In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from skeleton import skeleton
from matplotlib.animation import FFMpegWriter
import cv2 
from math import atan2
from scipy import ndimage
import src.utils as utils
from tqdm import tqdm

home_dir = '/data/LiftFly3D/prism/'
data_dir = '/mnt/NAS/SG/prism_data/'

border_width = 105
threshold = 45
bbox_width = 550
horiz_crop = 440

In [2]:
crop_positions = ['/bottom_view/videos/crop_loc_191125_PR_Fly1_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_004_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_004_prism.txt']

im_full = ['191125_PR/Fly1/001_prism/behData/images/',
           '191125_PR/Fly1/002_prism/behData/images/',
           '191125_PR/Fly1/003_prism/behData/images/',
           '191125_PR/Fly1/004_prism/behData/images/',
           '191125_PR/Fly2/001_prism/behData/images/',
           '191125_PR/Fly2/002_prism/behData/images/',
           '191125_PR/Fly2/003_prism/behData/images/',
           '191125_PR/Fly2/004_prism/behData/images/']

images_side = ['191125_PR/Fly1/001_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/002_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/003_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly1/004_prism/behData/images/side_view_prism_data_191125_PR_Fly1/',
               '191125_PR/Fly2/001_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/002_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/003_prism/behData/images/side_view_prism_data_191125_PR_Fly2/',
               '191125_PR/Fly2/004_prism/behData/images/side_view_prism_data_191125_PR_Fly2/']

images_bottom =  ['191125_PR/Fly1/001_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/002_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/003_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly1/004_prism/behData/images/bottom_view_prism_data_191125_PR_Fly1/',
                  '191125_PR/Fly2/001_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/002_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/003_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/',
                  '191125_PR/Fly2/004_prism/behData/images/bottom_view_prism_data_191125_PR_Fly2/']

DLC_predictions = ['191125_PR_Fly1_001_prism.pkl',
                  '191125_PR_Fly1_002_prism.pkl',
                  '191125_PR_Fly1_003_prism.pkl',
                  '191125_PR_Fly1_004_prism.pkl',
                  '191125_PR_Fly2_001_prism.pkl',
                  '191125_PR_Fly2_002_prism.pkl',
                  '191125_PR_Fly2_003_prism.pkl',
                  '191125_PR_Fly2_004_prism.pkl']

In [3]:
#crop loc file
def plot_skeleton(x, y, color_edge,  ax=None, good_keypts=None):
           
    for i, j in enumerate(G.edges()): 
        if good_keypts is not None:
            if (good_keypts[j[0]]==0) | (good_keypts[j[1]]==0):
                continue   
       
        u = np.array((x[j[0]], x[j[1]]))
        v = np.array((y[j[0]], y[j[1]]))
        if ax is not None:
            ax.plot(u, v, c=color_edge[j[0]], alpha=1.0, linewidth = 2)
        else:
            plt.plot(u, v, c=color_edge[j[0]], alpha=1.0, linewidth = 2)     

In [4]:
#load DLC predictions
which = 0
#poses = pickle.load(open(home_dir + 'data_unoriented/' + DLC_predictions[which], 'rb'))
poses = pickle.load(open(home_dir + DLC_predictions[which], 'rb'))
im_file, x_pos = utils.read_crop_pos(home_dir + crop_positions[which])
index = poses['index']
flip_idx = poses['flip_idx']
good_keypts = np.array(poses['good_keypts'])
G, color_edge = skeleton()

In [5]:
len(index)

1576

In [ ]:
#cropped image
i = 917

bottom = poses['points2d'][0,i,:,:]
side = poses['points2d'][1,i,:,:]

im_crop_bottom = cv2.imread(data_dir + images_bottom[which] + im_file[index[i]],cv2.IMREAD_GRAYSCALE)
im_crop_side = cv2.imread(data_dir + images_side[which] + im_file[index[i]],cv2.IMREAD_GRAYSCALE)

bottom_x = bottom[:,0]
bottom_y = bottom[:,1]
side_x = side[:,0]
side_y = side[:,1]

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True, figsize=(20,10))

ax1.imshow(im_crop_bottom,cmap='gray', vmin=0, vmax=255)
ax2.imshow(im_crop_side,cmap='gray', vmin=0, vmax=255)
ax1.set_axis_off()
ax2.set_axis_off()

plot_skeleton(bottom_x, bottom_y, color_edge, ax1)
plot_skeleton(side_x, side_y, color_edge, ax2, good_keypts[i,:])
#plt.savefig('DLC_pred.svg')

In [ ]:
#full image
fig = plt.figure(figsize=(15,6))
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)

bottom = poses['points2d'][0,:,:,:]
bottom = np.reshape(bottom, (bottom.shape[0], bottom.shape[1]*bottom.shape[2]))
side = poses['points2d'][1,:,:,:]
side = np.reshape(side, (side.shape[0], side.shape[1]*side.shape[2]))
            
writer = FFMpegWriter(fps=5)
with writer.saving(fig, "DLC_prediction.mp4", 50):
    for i in tqdm(range(1576)):  
        plt.cla()
        
        bottom_x, bottom_y = bottom[i,::2], bottom[i,1::2]
        side_x, side_y = side[i,::2], side[i,1::2]

        im = plt.imread(data_dir + im_full[which] + im_file[index[i]])

        plt.imshow(im,cmap='gray', vmin=0, vmax=255)
        plt.axhline(horiz_crop, color="white", linestyle="--")
        plt.axvline(border_width, color="white", linestyle="--")
        plt.axvline(im.shape[1] - border_width, color="white", linestyle="--")
                 
        plot_skeleton(border_width + bottom_x + int(x_pos[index[i]]), bottom_y, color_edge)
        plot_skeleton(border_width + side_x + int(x_pos[index[i]]), horiz_crop + side_y, color_edge, good_keypts=good_keypts[i,:])
        
        plt.text(120, 80, str(i), fontsize=50, color='white')
        
        plt.axis('off')
        writer.grab_frame()    


 16%|█▌        | 256/1576 [00:17<01:34, 14.03it/s]

In [ ]:
def video_to_imgs(vid_path):
    '''Convert video to a list of images'''
    
    cap = cv2.VideoCapture(vid_path)         
    imgs = []            
    while True:
        flag, frame = cap.read()
        if flag:                            
            imgs.append(frame)       
        else:
            break        
    
    return imgs


def imgs_to_video(img_paths, fps, out_path):
    '''Write video from a list of images'''
    
    imgs = []
    for i in img_paths:
        imgs.append(cv2.imread(i))
        
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_path, fourcc, fps, imgs[0].shape[:2])
    for img in imgs:
        out.write(img)
    out.release()

In [ ]:
imgs = video_to_imgs('output.mp4')
for i, im in enumerate(imgs):
    cv2.imwrite(str(i) + '.png', im)